### Convert Codebooks

#### Initialize Libraries and Global Variables

In [5]:
import re
import pandas as pd
import pickle
import psycopg2
import psycopg2.extras
import os

#### Batch Control

In [6]:
bcfcsv = 'codebook_batches_15.csv'
bcf = pd.read_csv(r'D:\\Documents\\conversion\\1_data import\\'+bcfcsv)
varlist = bcf.columns.tolist()
batchproc = bcf['batch'].tolist()
#batchproc = [1]
dctp = bcf.loc[bcf['batch'].isin(batchproc)].to_dict(orient='records')
print(dctp)

[{'batch': 15, 'desc': 'w tennessee', 'path': 'D:\\Documents\\datafiles\\batch15 - w tenn', 'cbkfilename': 'usa_00026.cbk', 'cbkname': 'cbk15', 'datafile': 'usa_00026.dat', 'cendata': 'cen15', 'cbkcol': 'variable,basis,range,length', 'pattern': '\\s{2}(\\w+)\\s+([HP])\\s+([0-9,-]+)\\s+(\\d+)'}]


Codebook Processing Functions

In [7]:
def read_codebook_text(cbkfile, pattern):
    with open(cbkfile, 'r') as f:
        text = f.read()       
        cbkdat = re.findall(pattern, text)
    return cbkdat

def codebook_to_df(cbkdat, cbkcols):
    cbk = pd.DataFrame.from_records(cbkdat,columns=cbkcols)
    return cbk

def parse_codebook(cbdf):
    a = cbdf['range'].str.replace('-',',').str.split(',').tolist()
    b = [[int(i) for i in j] for j in a]
    c = []
    for i in b:
        i[0] = i[0]-1
        c.append(i)
    #print(c)
    return c

def slice_codebook(c):
    sl=[]
    for i in c:
        if len(i) == 1: i.append(i[0]+1)
        s = slice(i[0],i[1])  
        sl.append(s)
    print('Batch:',batch,'Length of data:',len(cbdf), len(c),len(sl))
    return sl

def save_codebook(cbdf, c, sl, cbkpickle):   
    dfx = pd.DataFrame(c)
    dfx.columns = ['start_idx','end_idx']
    dfz = pd.DataFrame(sl)
    dfz.columns = ['slice_obj']
    dfy = pd.concat([cbdf, dfx[['start_idx','end_idx']],dfz[['slice_obj']]], axis=1)
    dfy.to_pickle(cbkpickle)
    
def read_codebook(cbkpickle):
    cbdf = pd.read_pickle(cbkpickle)
    #print('column slice_obj is ',type(a[0]),'\n')
    #print('Saved codebook',batch)
    return cbdf


### Codebook Processing

In [8]:
for batch in dctp:
    vd = batch
    batch = vd['batch']
    path = vd['path']+'\\'
    cbkfile = vd['path']+'\\'+vd['cbkfilename']
    cbkcols = vd['cbkcol'].split(',')
    cendata = vd['path']+'\\'+vd['cendata']
    pattern = vd['pattern']
    cbkpickle = vd['path']+'\\'+vd['cbkname']+'.pickle'
    cbkdat = read_codebook_text(cbkfile, pattern)
    cbdf = codebook_to_df(cbkdat, cbkcols)
    c = parse_codebook(cbdf)
    sl = slice_codebook(c)
    save_codebook(cbdf, c, sl, cbkpickle)
    

Batch: 15 Length of data: 79 79 79
